# App Rating Predictor
## Building an app rating predictor using data scraped from the android market

Import packages

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
pd.set_option('max_columns', None) #Show all columns, or else some will just be "..."

In [2]:
df = pd.read_csv("./input/google_app_scrap.csv")
df.head()

,Unnamed: 0,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,CURRENCY,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,FAMILY_GENRE,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
0,0,Skincare Routine,4.510000,Beauty,topselling_paid,609.0,34,11,23,65,465,89.0,"10,000+",10000,17616,False,3.99,USD,16M,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,NaN,False,False,False,"Aug 13, 2018",14/2/22,8/3/22
1,1,Booksy Biz: For Businesses,4.313544,Beauty,topgrossing,13176.0,1587,365,464,659,10095,1920.0,"1,000,000+",1000000,1270756,True,0.00,USD,24M,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Teen,NaN,False,True,False,"May 22, 2014",3/3/22,8/3/22
2,2,Perfect365 Makeup Photo Editor,4.295216,Beauty,topgrossing,782877.0,62917,22800,55202,121258,520690,49923.0,"50,000,000+",50000000,54737806,True,0.00,USD,201M,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Everyone,NaN,True,True,False,"Feb 19, 2013",4/3/22,8/3/22
3,3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,topgrossing,95.0,38,6,0,2,47,29.0,"10,000+",10000,29466,True,0.00,USD,67M,Glowbe LTD,"4103, Limassol Cyprus",Everyone,NaN,False,True,False,"Dec 2, 2020",20/2/22,8/3/22
4,4,PhiApp,4.233333,Beauty,topgrossing,916.0,122,15,61,45,671,26.0,"100,000+",100000,302978,True,0.00,USD,Varies with device,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Everyone,NaN,False,True,False,"Mar 13, 2015",20/1/22,8/3/22


## Data Cleaning

Basic statistical analysis with scraped data

In [3]:
df.describe(include="all")

,Unnamed: 0,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,CURRENCY,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,FAMILY_GENRE,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
count,23952.00000,23952,23934.000000,23952,23952,2.393400e+04,2.395200e+04,2.395200e+04,2.395200e+04,2.395200e+04,2.395200e+04,2.393400e+04,23952,2.395200e+04,2.395200e+04,23952,23952.000000,23952,23952,23952,19420,23952,0.0,23952,23952,23952,23762,23952,23952
unique,NaN,18496,NaN,48,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,NaN,NaN,2,NaN,1,666,12343,8614,5,NaN,2,2,2,3935,1600,1
top,NaN,Solitaire,NaN,Sports,topselling_free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,000,000+",NaN,NaN,True,NaN,USD,Varies with device,My Town Games Ltd,"Ha-Rekhev St 1, Tel Aviv-Yafo",Everyone,NaN,False,True,False,"Feb 21, 2022",4/3/22,8/3/22
freq,NaN,27,NaN,944,10593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4427,NaN,NaN,18520,NaN,23952,3170,137,137,17530,NaN,12182,14509,23439,44,955,23952
mean,11975.50000,NaN,3.988836,NaN,NaN,3.899129e+05,4.401561e+04,9.365627e+03,1.810578e+04,4.228248e+04,2.758255e+05,1.778189e+04,NaN,1.891777e+07,3.073138e+07,NaN,1.422892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,6914.49116,NaN,1.044631,NaN,NaN,3.676933e+06,7.961011e+05,1.007322e+05,1.726730e+05,3.569045e+05,2.428996e+06,1.179491e+05,NaN,2.201441e+08,2.802337e+08,NaN,8.832669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.00000,NaN,0.000000,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,5987.75000,NaN,3.891892,NaN,NaN,6.412500e+02,5.800000e+01,1.200000e+01,2.300000e+01,5.100000e+01,3.927500e+02,6.400000e+01,NaN,5.000000e+04,5.644300e+04,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,11975.50000,NaN,4.300697,NaN,NaN,8.450000e+03,6.805000e+02,1.900000e+02,3.470000e+02,7.885000e+02,5.698000e+03,5.750000e+02,NaN,5.000000e+05,8.288390e+05,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,17963.25000,NaN,4.561213,NaN,NaN,7.308575e+04,5.809500e+03,1.719250e+03,3.201000e+03,7.650000e+03,5.187900e+04,4.237750e+03,NaN,5.000000e+06,7.747612e+06,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23952 entries, 0 to 23951
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         23952 non-null  int64  
 1   APP_NAME           23952 non-null  object 
 2   RATING             23934 non-null  float64
 3   CATEGORY           23952 non-null  object 
 4   COLLECTIONS        23952 non-null  object 
 5   RATING_COUNT       23934 non-null  float64
 6   1_STAR_RATINGS     23952 non-null  int64  
 7   2_STAR_RATINGS     23952 non-null  int64  
 8   3_STAR_RATINGS     23952 non-null  int64  
 9   4_STAR_RATINGS     23952 non-null  int64  
 10  5_STAR_RATINGS     23952 non-null  int64  
 11  REVIEW_COUNT       23934 non-null  float64
 12  INSTALLS           23952 non-null  object 
 13  MIN_INSTALLS       23952 non-null  int64  
 14  MAX_INSTALLS       23952 non-null  int64  
 15  FREE               23952 non-null  bool   
 16  PRICE              239

Some initial observations
1. UNNAMED: 0 column is same as the index
2. There are no values in FAMILY_GENRE
3. All the values in CURRENCY are the same (usd) which indicates the currency of the price of the app
4. The 5 n_STAR_RATING columns does not always sum up to RATING_COUNT
5. SIZE contains string objects with postfix of K, M, G to indicate the size of the app in bytes
6. The formats for columns with date objects are inconsistent

### REMOVE USELESS COLUMNS

First, we remove columns "Unnamed: 0". "FAMILY_GENRE", "CURRENCY" as they contain same/empty values which will be useless in eda

In [5]:
df = df.drop(columns=['Unnamed: 0', 'FAMILY_GENRE', 'CURRENCY'])
df.columns

Index(['APP_NAME', 'RATING', 'CATEGORY', 'COLLECTIONS', 'RATING_COUNT',
       '1_STAR_RATINGS', '2_STAR_RATINGS', '3_STAR_RATINGS', '4_STAR_RATINGS',
       '5_STAR_RATINGS', 'REVIEW_COUNT', 'INSTALLS', 'MIN_INSTALLS',
       'MAX_INSTALLS', 'FREE', 'PRICE', 'SIZE', 'DEVELOPER',
       'DEVELOPER_ADDRESS', 'CONTENT_RATING', 'AD_SUPPORTED',
       'IN_APP_PURCHASES', 'EDITORS_CHOICE', 'RELEASED_DATE', 'LAST_UPDATED',
       'DAY_SCRAPED'],
      dtype='object')

### Cleaning RATING_COUNT
The RATING_COUNT column has inconsistent values, change the values in RATING_COUNT column to the sum of 1_STAR_RATING, 2_STAR_RATING, 3_STAR_RATING, 4_STAR_RATING, 5_STAR_RATING columns

In [6]:
df['RATING_COUNT'] = df[['1_STAR_RATINGS', '2_STAR_RATINGS', '3_STAR_RATINGS', '4_STAR_RATINGS', '5_STAR_RATINGS']].sum(axis=1)
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
0,Skincare Routine,4.510000,Beauty,topselling_paid,598,34,11,23,65,465,89.0,"10,000+",10000,17616,False,3.99,16M,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,False,False,False,"Aug 13, 2018",14/2/22,8/3/22
1,Booksy Biz: For Businesses,4.313544,Beauty,topgrossing,13170,1587,365,464,659,10095,1920.0,"1,000,000+",1000000,1270756,True,0.00,24M,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Teen,False,True,False,"May 22, 2014",3/3/22,8/3/22
2,Perfect365 Makeup Photo Editor,4.295216,Beauty,topgrossing,782867,62917,22800,55202,121258,520690,49923.0,"50,000,000+",50000000,54737806,True,0.00,201M,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Everyone,True,True,False,"Feb 19, 2013",4/3/22,8/3/22
3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,topgrossing,93,38,6,0,2,47,29.0,"10,000+",10000,29466,True,0.00,67M,Glowbe LTD,"4103, Limassol Cyprus",Everyone,False,True,False,"Dec 2, 2020",20/2/22,8/3/22
4,PhiApp,4.233333,Beauty,topgrossing,914,122,15,61,45,671,26.0,"100,000+",100000,302978,True,0.00,Varies with device,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Everyone,False,True,False,"Mar 13, 2015",20/1/22,8/3/22


### Cleaning SIZE
SIZE contains string objects, convert them to number for eda later. <br/>
For example: 12M -> 12000000, 12K -> 12000, varies with device -> NaN (handle this later)

In [7]:
df["SIZE"] = df["SIZE"].str.replace("Varies with device","-1") #replacing "Varies with device" with -1 
df["SIZE"] = df["SIZE"].str.replace(",","") #replacing ',' with nothing

In [8]:
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
0,Skincare Routine,4.510000,Beauty,topselling_paid,598,34,11,23,65,465,89.0,"10,000+",10000,17616,False,3.99,16M,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,False,False,False,"Aug 13, 2018",14/2/22,8/3/22
1,Booksy Biz: For Businesses,4.313544,Beauty,topgrossing,13170,1587,365,464,659,10095,1920.0,"1,000,000+",1000000,1270756,True,0.00,24M,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Teen,False,True,False,"May 22, 2014",3/3/22,8/3/22
2,Perfect365 Makeup Photo Editor,4.295216,Beauty,topgrossing,782867,62917,22800,55202,121258,520690,49923.0,"50,000,000+",50000000,54737806,True,0.00,201M,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Everyone,True,True,False,"Feb 19, 2013",4/3/22,8/3/22
3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,topgrossing,93,38,6,0,2,47,29.0,"10,000+",10000,29466,True,0.00,67M,Glowbe LTD,"4103, Limassol Cyprus",Everyone,False,True,False,"Dec 2, 2020",20/2/22,8/3/22
4,PhiApp,4.233333,Beauty,topgrossing,914,122,15,61,45,671,26.0,"100,000+",100000,302978,True,0.00,-1,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Everyone,False,True,False,"Mar 13, 2015",20/1/22,8/3/22


In [9]:
#changing k to (10**3), M to (10**6) and G to (10**9)
df["SIZE"] = (df["SIZE"].replace(r'[kMG]+$', '', regex=True).astype(float) * \
                   df["SIZE"].str.extract(r'[\d\.]+([kMG]+)', expand=False)
                   .fillna(1)
                   .replace(['k','M','G'], [10**3, 10**6,10**9]).astype(int))
df['SIZE'] = df['SIZE'].astype('int64') #converting float to int64
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
0,Skincare Routine,4.510000,Beauty,topselling_paid,598,34,11,23,65,465,89.0,"10,000+",10000,17616,False,3.99,16000000,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,False,False,False,"Aug 13, 2018",14/2/22,8/3/22
1,Booksy Biz: For Businesses,4.313544,Beauty,topgrossing,13170,1587,365,464,659,10095,1920.0,"1,000,000+",1000000,1270756,True,0.00,24000000,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Teen,False,True,False,"May 22, 2014",3/3/22,8/3/22
2,Perfect365 Makeup Photo Editor,4.295216,Beauty,topgrossing,782867,62917,22800,55202,121258,520690,49923.0,"50,000,000+",50000000,54737806,True,0.00,201000000,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Everyone,True,True,False,"Feb 19, 2013",4/3/22,8/3/22
3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,topgrossing,93,38,6,0,2,47,29.0,"10,000+",10000,29466,True,0.00,67000000,Glowbe LTD,"4103, Limassol Cyprus",Everyone,False,True,False,"Dec 2, 2020",20/2/22,8/3/22
4,PhiApp,4.233333,Beauty,topgrossing,914,122,15,61,45,671,26.0,"100,000+",100000,302978,True,0.00,-1,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Everyone,False,True,False,"Mar 13, 2015",20/1/22,8/3/22


Convert all -1 values to NaN, we will fill this later

In [10]:
df.loc[df['SIZE'] == -1, 'SIZE'] = np.nan
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
0,Skincare Routine,4.510000,Beauty,topselling_paid,598,34,11,23,65,465,89.0,"10,000+",10000,17616,False,3.99,16000000.0,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,False,False,False,"Aug 13, 2018",14/2/22,8/3/22
1,Booksy Biz: For Businesses,4.313544,Beauty,topgrossing,13170,1587,365,464,659,10095,1920.0,"1,000,000+",1000000,1270756,True,0.00,24000000.0,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Teen,False,True,False,"May 22, 2014",3/3/22,8/3/22
2,Perfect365 Makeup Photo Editor,4.295216,Beauty,topgrossing,782867,62917,22800,55202,121258,520690,49923.0,"50,000,000+",50000000,54737806,True,0.00,201000000.0,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Everyone,True,True,False,"Feb 19, 2013",4/3/22,8/3/22
3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,topgrossing,93,38,6,0,2,47,29.0,"10,000+",10000,29466,True,0.00,67000000.0,Glowbe LTD,"4103, Limassol Cyprus",Everyone,False,True,False,"Dec 2, 2020",20/2/22,8/3/22
4,PhiApp,4.233333,Beauty,topgrossing,914,122,15,61,45,671,26.0,"100,000+",100000,302978,True,0.00,NaN,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Everyone,False,True,False,"Mar 13, 2015",20/1/22,8/3/22


### Cleaning DAY_RELEASED
The format of dates in DAY_RELEASED, LAST_UPDATED and DAY_SCRAPED are inconsistent, convert both of them to the format "dd/mm/yyyy"

In [11]:
df['RELEASED_DATE'] = pd.to_datetime(df['RELEASED_DATE'])
df['LAST_UPDATED'] = pd.to_datetime(df['LAST_UPDATED'], format="%d/%m/%y")
df['DAY_SCRAPED'] = pd.to_datetime(df['DAY_SCRAPED'], format="%d/%m/%y")
df['RELEASED_DATE'] = df['RELEASED_DATE'].dt.strftime('%d/%m/%Y')
df['LAST_UPDATED'] = df['LAST_UPDATED'].dt.strftime('%d/%m/%Y')
df['DAY_SCRAPED'] = df['DAY_SCRAPED'].dt.strftime('%d/%m/%Y')
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
0,Skincare Routine,4.510000,Beauty,topselling_paid,598,34,11,23,65,465,89.0,"10,000+",10000,17616,False,3.99,16000000.0,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,False,False,False,13/08/2018,14/02/2022,08/03/2022
1,Booksy Biz: For Businesses,4.313544,Beauty,topgrossing,13170,1587,365,464,659,10095,1920.0,"1,000,000+",1000000,1270756,True,0.00,24000000.0,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Teen,False,True,False,22/05/2014,03/03/2022,08/03/2022
2,Perfect365 Makeup Photo Editor,4.295216,Beauty,topgrossing,782867,62917,22800,55202,121258,520690,49923.0,"50,000,000+",50000000,54737806,True,0.00,201000000.0,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Everyone,True,True,False,19/02/2013,04/03/2022,08/03/2022
3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,topgrossing,93,38,6,0,2,47,29.0,"10,000+",10000,29466,True,0.00,67000000.0,Glowbe LTD,"4103, Limassol Cyprus",Everyone,False,True,False,02/12/2020,20/02/2022,08/03/2022
4,PhiApp,4.233333,Beauty,topgrossing,914,122,15,61,45,671,26.0,"100,000+",100000,302978,True,0.00,NaN,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Everyone,False,True,False,13/03/2015,20/01/2022,08/03/2022


### Dealing with NULL Values

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23952 entries, 0 to 23951
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   APP_NAME           23952 non-null  object 
 1   RATING             23934 non-null  float64
 2   CATEGORY           23952 non-null  object 
 3   COLLECTIONS        23952 non-null  object 
 4   RATING_COUNT       23952 non-null  int64  
 5   1_STAR_RATINGS     23952 non-null  int64  
 6   2_STAR_RATINGS     23952 non-null  int64  
 7   3_STAR_RATINGS     23952 non-null  int64  
 8   4_STAR_RATINGS     23952 non-null  int64  
 9   5_STAR_RATINGS     23952 non-null  int64  
 10  REVIEW_COUNT       23934 non-null  float64
 11  INSTALLS           23952 non-null  object 
 12  MIN_INSTALLS       23952 non-null  int64  
 13  MAX_INSTALLS       23952 non-null  int64  
 14  FREE               23952 non-null  bool   
 15  PRICE              23952 non-null  float64
 16  SIZE               207

By observation and previous cleaning steps, there are 6 columns with NULL values which are:
- REVIEW_COUNT
- RATING
- RATING_COUNT
- RELEASED_DATE
- SIZE
- DEVELOPER_ADDRESS


For "REVIEW_COUNT", "RATING", "RATING_COUNT", the values are null value most likely because the app is too new and no customers have done any review on them, so remove them from the dataset. 

In [13]:
df = df.dropna(axis=0, subset=['REVIEW_COUNT', 'RATING', 'RATING_COUNT'])
df = df.reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23934 entries, 0 to 23933
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              23934 non-null  int64  
 1   APP_NAME           23934 non-null  object 
 2   RATING             23934 non-null  float64
 3   CATEGORY           23934 non-null  object 
 4   COLLECTIONS        23934 non-null  object 
 5   RATING_COUNT       23934 non-null  int64  
 6   1_STAR_RATINGS     23934 non-null  int64  
 7   2_STAR_RATINGS     23934 non-null  int64  
 8   3_STAR_RATINGS     23934 non-null  int64  
 9   4_STAR_RATINGS     23934 non-null  int64  
 10  5_STAR_RATINGS     23934 non-null  int64  
 11  REVIEW_COUNT       23934 non-null  float64
 12  INSTALLS           23934 non-null  object 
 13  MIN_INSTALLS       23934 non-null  int64  
 14  MAX_INSTALLS       23934 non-null  int64  
 15  FREE               23934 non-null  bool   
 16  PRICE              239

As for "RELEASED_DATE", fill the null values with the median. 

But first we create to new columns to replace "RELEASED_DATE" and "LAST_UPDATED" to contain numerical values instead of Date object.

In [14]:
for i in range(df.shape[0]):
    if (pd.isnull(df.loc[i, "LAST_UPDATED"])):
        df.loc[i, "DAYS_SINCE_UPDATE"] = np.nan
        continue
    date_from = pd.to_datetime(df.loc[i, "LAST_UPDATED"], format="%d/%m/%Y").date()
    date_till = pd.to_datetime(df.loc[i, "DAY_SCRAPED"], format="%d/%m/%Y").date()
    date_elasped = np.int64((date_till-date_from).days)
    df.loc[i, "DAYS_SINCE_UPDATE"] = date_elasped
    
    if (pd.isnull(df.loc[i, "RELEASED_DATE"])):
        df.loc[i, "DAYS_SINCE_RELEASED"] = np.nan
        continue
    
    date_from = pd.to_datetime(df.loc[i, "RELEASED_DATE"], format="%d/%m/%Y").date()
    date_till = pd.to_datetime(df.loc[i, "DAY_SCRAPED"], format="%d/%m/%Y").date()
    date_elasped = np.int64((date_till-date_from).days)
    df.loc[i, "DAYS_SINCE_RELEASED"] = date_elasped
    
df.head()

,index,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
0,0,Skincare Routine,4.510000,Beauty,topselling_paid,598,34,11,23,65,465,89.0,"10,000+",10000,17616,False,3.99,16000000.0,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,False,False,False,13/08/2018,14/02/2022,08/03/2022,22.0,1303.0
1,1,Booksy Biz: For Businesses,4.313544,Beauty,topgrossing,13170,1587,365,464,659,10095,1920.0,"1,000,000+",1000000,1270756,True,0.00,24000000.0,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Teen,False,True,False,22/05/2014,03/03/2022,08/03/2022,5.0,2847.0
2,2,Perfect365 Makeup Photo Editor,4.295216,Beauty,topgrossing,782867,62917,22800,55202,121258,520690,49923.0,"50,000,000+",50000000,54737806,True,0.00,201000000.0,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Everyone,True,True,False,19/02/2013,04/03/2022,08/03/2022,4.0,3304.0
3,3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,topgrossing,93,38,6,0,2,47,29.0,"10,000+",10000,29466,True,0.00,67000000.0,Glowbe LTD,"4103, Limassol Cyprus",Everyone,False,True,False,02/12/2020,20/02/2022,08/03/2022,16.0,461.0
4,4,PhiApp,4.233333,Beauty,topgrossing,914,122,15,61,45,671,26.0,"100,000+",100000,302978,True,0.00,NaN,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Everyone,False,True,False,13/03/2015,20/01/2022,08/03/2022,47.0,2552.0


Fill na values with the median

In [15]:
median_dsr = df["DAYS_SINCE_RELEASED"].median()
median_dsu = df["DAYS_SINCE_UPDATE"].median()
df["DAYS_SINCE_RELEASED"].fillna(median_dsr, inplace=True)
df["DAYS_SINCE_UPDATE"].fillna(median_dsu, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23934 entries, 0 to 23933
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                23934 non-null  int64  
 1   APP_NAME             23934 non-null  object 
 2   RATING               23934 non-null  float64
 3   CATEGORY             23934 non-null  object 
 4   COLLECTIONS          23934 non-null  object 
 5   RATING_COUNT         23934 non-null  int64  
 6   1_STAR_RATINGS       23934 non-null  int64  
 7   2_STAR_RATINGS       23934 non-null  int64  
 8   3_STAR_RATINGS       23934 non-null  int64  
 9   4_STAR_RATINGS       23934 non-null  int64  
 10  5_STAR_RATINGS       23934 non-null  int64  
 11  REVIEW_COUNT         23934 non-null  float64
 12  INSTALLS             23934 non-null  object 
 13  MIN_INSTALLS         23934 non-null  int64  
 14  MAX_INSTALLS         23934 non-null  int64  
 15  FREE                 23934 non-null 

In [16]:
df['DAYS_SINCE_RELEASED'] = df['DAYS_SINCE_RELEASED'].astype('int64')
df['DAYS_SINCE_UPDATE'] = df['DAYS_SINCE_UPDATE'].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23934 entries, 0 to 23933
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                23934 non-null  int64  
 1   APP_NAME             23934 non-null  object 
 2   RATING               23934 non-null  float64
 3   CATEGORY             23934 non-null  object 
 4   COLLECTIONS          23934 non-null  object 
 5   RATING_COUNT         23934 non-null  int64  
 6   1_STAR_RATINGS       23934 non-null  int64  
 7   2_STAR_RATINGS       23934 non-null  int64  
 8   3_STAR_RATINGS       23934 non-null  int64  
 9   4_STAR_RATINGS       23934 non-null  int64  
 10  5_STAR_RATINGS       23934 non-null  int64  
 11  REVIEW_COUNT         23934 non-null  float64
 12  INSTALLS             23934 non-null  object 
 13  MIN_INSTALLS         23934 non-null  int64  
 14  MAX_INSTALLS         23934 non-null  int64  
 15  FREE                 23934 non-null 

Drop RELEASED_DATE, LAST_UPDATED and DAY_SCRAPED because we dont need them anymore

In [17]:
df = df.drop(columns=['RELEASED_DATE', 'LAST_UPDATED', 'DAY_SCRAPED'])
df.columns

Index(['index', 'APP_NAME', 'RATING', 'CATEGORY', 'COLLECTIONS',
       'RATING_COUNT', '1_STAR_RATINGS', '2_STAR_RATINGS', '3_STAR_RATINGS',
       '4_STAR_RATINGS', '5_STAR_RATINGS', 'REVIEW_COUNT', 'INSTALLS',
       'MIN_INSTALLS', 'MAX_INSTALLS', 'FREE', 'PRICE', 'SIZE', 'DEVELOPER',
       'DEVELOPER_ADDRESS', 'CONTENT_RATING', 'AD_SUPPORTED',
       'IN_APP_PURCHASES', 'EDITORS_CHOICE', 'DAYS_SINCE_UPDATE',
       'DAYS_SINCE_RELEASED'],
      dtype='object')

Filling null values of SIZE with median

In [18]:
median_size = df["SIZE"].median()
df["SIZE"].fillna(median_size, inplace=True)
df["SIZE"]

0         16000000.0
1         24000000.0
2        201000000.0
3         67000000.0
4         45000000.0
            ...     
23929     57000000.0
23930     60000000.0
23931    126000000.0
23932    101000000.0
23933      7500000.0
Name: SIZE, Length: 23934, dtype: float64

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23934 entries, 0 to 23933
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                23934 non-null  int64  
 1   APP_NAME             23934 non-null  object 
 2   RATING               23934 non-null  float64
 3   CATEGORY             23934 non-null  object 
 4   COLLECTIONS          23934 non-null  object 
 5   RATING_COUNT         23934 non-null  int64  
 6   1_STAR_RATINGS       23934 non-null  int64  
 7   2_STAR_RATINGS       23934 non-null  int64  
 8   3_STAR_RATINGS       23934 non-null  int64  
 9   4_STAR_RATINGS       23934 non-null  int64  
 10  5_STAR_RATINGS       23934 non-null  int64  
 11  REVIEW_COUNT         23934 non-null  float64
 12  INSTALLS             23934 non-null  object 
 13  MIN_INSTALLS         23934 non-null  int64  
 14  MAX_INSTALLS         23934 non-null  int64  
 15  FREE                 23934 non-null 

The raw address of developers might be hard to use for eda, instead generate a new feature "COUNTRY" that holds the country the developer originated.

First, import the dataset of predicted country of each developer address that we generated using google maps api.

In [20]:
country_df = pd.read_csv("./input/developer_country.csv")
country_df.head()

,index,DEVELOPER_ADDRESS,COUNTRY
0,0,23 Stoneleigh Avenue\nBN1 8NP,United Kingdom
1,1,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Poland
2,2,"Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Usa
3,3,"4103, Limassol Cyprus",Cyprus
4,4,"Gartengasse 8/8\n1050, Vienna\nAustria",Austria


Remove # from each row in DEVELOPER_ADDRESS column to match the format of mapping in the country_df

In [21]:
df = df.drop(["index"], axis = 1)
for i in range(df.shape[0]):
    developer_addr = df.loc[i, "DEVELOPER_ADDRESS"]
    if (pd.isnull(developer_addr)):
        continue
    df.loc[i, "DEVELOPER_ADDRESS"] = developer_addr.replace("#", " ")
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
0,Skincare Routine,4.510000,Beauty,topselling_paid,598,34,11,23,65,465,89.0,"10,000+",10000,17616,False,3.99,16000000.0,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,False,False,False,22,1303
1,Booksy Biz: For Businesses,4.313544,Beauty,topgrossing,13170,1587,365,464,659,10095,1920.0,"1,000,000+",1000000,1270756,True,0.00,24000000.0,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Teen,False,True,False,5,2847
2,Perfect365 Makeup Photo Editor,4.295216,Beauty,topgrossing,782867,62917,22800,55202,121258,520690,49923.0,"50,000,000+",50000000,54737806,True,0.00,201000000.0,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Everyone,True,True,False,4,3304
3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,topgrossing,93,38,6,0,2,47,29.0,"10,000+",10000,29466,True,0.00,67000000.0,Glowbe LTD,"4103, Limassol Cyprus",Everyone,False,True,False,16,461
4,PhiApp,4.233333,Beauty,topgrossing,914,122,15,61,45,671,26.0,"100,000+",100000,302978,True,0.00,45000000.0,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Everyone,False,True,False,47,2552


Map the developer address to its country 

In [22]:
df['COUNTRY']=df[['DEVELOPER_ADDRESS']].merge(country_df,how='left').COUNTRY
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED,COUNTRY
0,Skincare Routine,4.510000,Beauty,topselling_paid,598,34,11,23,65,465,89.0,"10,000+",10000,17616,False,3.99,16000000.0,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,False,False,False,22,1303,United Kingdom
1,Booksy Biz: For Businesses,4.313544,Beauty,topgrossing,13170,1587,365,464,659,10095,1920.0,"1,000,000+",1000000,1270756,True,0.00,24000000.0,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Teen,False,True,False,5,2847,Poland
2,Perfect365 Makeup Photo Editor,4.295216,Beauty,topgrossing,782867,62917,22800,55202,121258,520690,49923.0,"50,000,000+",50000000,54737806,True,0.00,201000000.0,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Everyone,True,True,False,4,3304,Usa
3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,topgrossing,93,38,6,0,2,47,29.0,"10,000+",10000,29466,True,0.00,67000000.0,Glowbe LTD,"4103, Limassol Cyprus",Everyone,False,True,False,16,461,Cyprus
4,PhiApp,4.233333,Beauty,topgrossing,914,122,15,61,45,671,26.0,"100,000+",100000,302978,True,0.00,45000000.0,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Everyone,False,True,False,47,2552,Austria


Reposition the country column

In [23]:
df.insert(19, 'COUNTRY', df.pop('COUNTRY'))
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,COUNTRY,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
0,Skincare Routine,4.510000,Beauty,topselling_paid,598,34,11,23,65,465,89.0,"10,000+",10000,17616,False,3.99,16000000.0,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,United Kingdom,Everyone,False,False,False,22,1303
1,Booksy Biz: For Businesses,4.313544,Beauty,topgrossing,13170,1587,365,464,659,10095,1920.0,"1,000,000+",1000000,1270756,True,0.00,24000000.0,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Poland,Teen,False,True,False,5,2847
2,Perfect365 Makeup Photo Editor,4.295216,Beauty,topgrossing,782867,62917,22800,55202,121258,520690,49923.0,"50,000,000+",50000000,54737806,True,0.00,201000000.0,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Usa,Everyone,True,True,False,4,3304
3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,topgrossing,93,38,6,0,2,47,29.0,"10,000+",10000,29466,True,0.00,67000000.0,Glowbe LTD,"4103, Limassol Cyprus",Cyprus,Everyone,False,True,False,16,461
4,PhiApp,4.233333,Beauty,topgrossing,914,122,15,61,45,671,26.0,"100,000+",100000,302978,True,0.00,45000000.0,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Austria,Everyone,False,True,False,47,2552


In [24]:
df["COUNTRY"].describe()

count     16263
unique      103
top         Usa
freq       5722
Name: COUNTRY, dtype: object

There are 16263 apps with predicted developer country, which means around 4000 apps do not have predicted country/registered country. 

We will only drop them in model building later.

In [25]:
df.sort_values("APP_NAME", inplace = True)
df.reset_index(inplace=True)
df.drop(["index"], axis=1, inplace=True)
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,COUNTRY,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
0,"""Ghost Voice Catcher"" AUTO EVP",3.933333,Lifestyle,topselling_paid,68,12,1,6,7,42,24.0,"1,000+",1000,3608,False,5.99,1100000.0,BIG BEARD Studios,114 1/2 Ash st. Nashua NH 03060,Usa,Everyone,False,False,False,1820,3482
1,"""OXXO""",4.740000,Puzzle,topselling_paid,975,23,0,15,94,843,54.0,"10,000+",10000,21680,False,0.99,81000000.0,Hamster On Coke Games,ul Jugosłowiańska 37F/10\n73-110 Stargard Szcz...,Poland,Everyone,False,False,False,0,945
2,#DRIVE,4.434152,Racing,topgrossing,229679,9352,4740,15249,47804,152534,6473.0,"5,000,000+",5000000,9119244,True,0.00,132000000.0,Pixel Perfect Dude S.A.,ul. Polna 28/6 lok. 4\n62-800 Kalisz\nPolska,NaN,Everyone,True,True,False,4,1084
3,#DRIVE,4.434152,Racing,topselling_free,229679,9352,4740,15249,47804,152534,6473.0,"5,000,000+",5000000,9119244,True,0.00,132000000.0,Pixel Perfect Dude S.A.,ul. Polna 28/6 lok. 4\n62-800 Kalisz\nPolska,NaN,Everyone,True,True,False,4,1084
4,#SelfCare,4.463476,Role Playing,topselling_free,14932,865,336,1088,1353,11290,1501.0,"1,000,000+",1000000,1209692,True,0.00,83000000.0,TRU LUV Inc.,"186-720 King Street West\nToronto, Ontario, Ca...",Canada,Teen,False,True,False,952,1275


In [26]:
temp_df = df.groupby("APP_NAME")["CATEGORY"].apply(list)
df.drop_duplicates(subset ="APP_NAME",
                     keep = "first", inplace = True)

In [27]:
df.reset_index(inplace=True)
df.drop(["index"], axis = 1, inplace= True)
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,COUNTRY,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
0,"""Ghost Voice Catcher"" AUTO EVP",3.933333,Lifestyle,topselling_paid,68,12,1,6,7,42,24.0,"1,000+",1000,3608,False,5.99,1100000.0,BIG BEARD Studios,114 1/2 Ash st. Nashua NH 03060,Usa,Everyone,False,False,False,1820,3482
1,"""OXXO""",4.740000,Puzzle,topselling_paid,975,23,0,15,94,843,54.0,"10,000+",10000,21680,False,0.99,81000000.0,Hamster On Coke Games,ul Jugosłowiańska 37F/10\n73-110 Stargard Szcz...,Poland,Everyone,False,False,False,0,945
2,#DRIVE,4.434152,Racing,topgrossing,229679,9352,4740,15249,47804,152534,6473.0,"5,000,000+",5000000,9119244,True,0.00,132000000.0,Pixel Perfect Dude S.A.,ul. Polna 28/6 lok. 4\n62-800 Kalisz\nPolska,NaN,Everyone,True,True,False,4,1084
3,#SelfCare,4.463476,Role Playing,topselling_free,14932,865,336,1088,1353,11290,1501.0,"1,000,000+",1000000,1209692,True,0.00,83000000.0,TRU LUV Inc.,"186-720 King Street West\nToronto, Ontario, Ca...",Canada,Teen,False,True,False,952,1275
4,#open Polyamorous + ENM Dating,2.550000,Dating,topselling_free,708,263,163,63,70,149,362.0,"50,000+",50000,93463,True,0.00,19000000.0,#open,NaN,NaN,Mature 17+,False,True,False,17,1026


In [28]:
i = 0
for tags in temp_df:
    to_add = []
    [to_add.append(x) for x in tags if x not in to_add]
    df.loc[i, "CATEGORY"] = str(to_add)
    i+=1
df.head() 

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,COUNTRY,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
0,"""Ghost Voice Catcher"" AUTO EVP",3.933333,['Lifestyle'],topselling_paid,68,12,1,6,7,42,24.0,"1,000+",1000,3608,False,5.99,1100000.0,BIG BEARD Studios,114 1/2 Ash st. Nashua NH 03060,Usa,Everyone,False,False,False,1820,3482
1,"""OXXO""",4.740000,['Puzzle'],topselling_paid,975,23,0,15,94,843,54.0,"10,000+",10000,21680,False,0.99,81000000.0,Hamster On Coke Games,ul Jugosłowiańska 37F/10\n73-110 Stargard Szcz...,Poland,Everyone,False,False,False,0,945
2,#DRIVE,4.434152,['Racing'],topgrossing,229679,9352,4740,15249,47804,152534,6473.0,"5,000,000+",5000000,9119244,True,0.00,132000000.0,Pixel Perfect Dude S.A.,ul. Polna 28/6 lok. 4\n62-800 Kalisz\nPolska,NaN,Everyone,True,True,False,4,1084
3,#SelfCare,4.463476,['Role Playing'],topselling_free,14932,865,336,1088,1353,11290,1501.0,"1,000,000+",1000000,1209692,True,0.00,83000000.0,TRU LUV Inc.,"186-720 King Street West\nToronto, Ontario, Ca...",Canada,Teen,False,True,False,952,1275
4,#open Polyamorous + ENM Dating,2.550000,['Dating'],topselling_free,708,263,163,63,70,149,362.0,"50,000+",50000,93463,True,0.00,19000000.0,#open,NaN,NaN,Mature 17+,False,True,False,17,1026


In [29]:
df = df[df["RATING_COUNT"]!= 0]
df.reset_index(inplace=True)
df.drop(["index"], axis = 1, inplace= True)
df

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,COUNTRY,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
0,"""Ghost Voice Catcher"" AUTO EVP",3.933333,['Lifestyle'],topselling_paid,68,12,1,6,7,42,24.0,"1,000+",1000,3608,False,5.99,1100000.0,BIG BEARD Studios,114 1/2 Ash st. Nashua NH 03060,Usa,Everyone,False,False,False,1820,3482
1,"""OXXO""",4.740000,['Puzzle'],topselling_paid,975,23,0,15,94,843,54.0,"10,000+",10000,21680,False,0.99,81000000.0,Hamster On Coke Games,ul Jugosłowiańska 37F/10\n73-110 Stargard Szcz...,Poland,Everyone,False,False,False,0,945
2,#DRIVE,4.434152,['Racing'],topgrossing,229679,9352,4740,15249,47804,152534,6473.0,"5,000,000+",5000000,9119244,True,0.00,132000000.0,Pixel Perfect Dude S.A.,ul. Polna 28/6 lok. 4\n62-800 Kalisz\nPolska,NaN,Everyone,True,True,False,4,1084
3,#SelfCare,4.463476,['Role Playing'],topselling_free,14932,865,336,1088,1353,11290,1501.0,"1,000,000+",1000000,1209692,True,0.00,83000000.0,TRU LUV Inc.,"186-720 King Street West\nToronto, Ontario, Ca...",Canada,Teen,False,True,False,952,1275
4,#open Polyamorous + ENM Dating,2.550000,['Dating'],topselling_free,708,263,163,63,70,149,362.0,"50,000+",50000,93463,True,0.00,19000000.0,#open,NaN,NaN,Mature 17+,False,True,False,17,1026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17465,투믹스,2.986667,['Comics'],topselling_free,6057,1777,646,1050,1050,1534,56.0,"1,000,000+",1000000,1834618,True,0.00,14000000.0,(주)투믹스,"서울특별시 강남구 테헤란로 98길 8, 4층(대치동, KT&G 대치타워)",NaN,Teen,False,True,False,12,1913
17466,피망 뉴맞고: 고스톱 대표 맞고 게임,3.840278,['Card'],topgrossing,352939,68709,19525,14582,46465,203658,691.0,"10,000,000+",10000000,10454514,True,0.00,85000000.0,NEOWIZ corp,경기도 성남시 분당구 대왕판교로 645번길 14 네오위즈판교타워,South Korea,Teen,True,True,False,21,3186
17467,허기 워기,4.200000,['Music'],topselling_free,717,143,0,0,0,574,5.0,"100,000+",100000,200717,True,0.00,49000000.0,Dark Horror Game,NaN,NaN,Everyone,True,False,False,100,101
17468,ﺷﺎﻫﺪ - Shahid,3.733209,['Entertainment'],topgrossing,630347,164347,17917,24095,39100,384888,6188.0,"50,000,000+",50000000,91701800,True,0.00,14000000.0,MBC Group,"DMC3,\nDubai Media City,\nDubai, UAE",United Arab Emirates,Everyone,True,True,False,1,2050


In [30]:
#exporting the cleaned data into a csv file
file_path = "./output/google_app_scrap_cleaned.csv"
df.to_csv(path_or_buf=file_path, index=False);

### EDA

Dev
1. Best developers and their top categories.
2. Developers that made the most apps.

#### Yifei
1. Does editor's choice affect ratings and installs?
2. Does size of the app affect total installs? (some people don't like to install large apps)
3. Does content rating, price(free / paid), ad supported apps has impact on the rating?
4. Does in game purchases affect rating
5. Does Day since updated affect rating
6. Does day since released affect installs (find apps that has less installs despite released for a long time)
7. How to get "High" Rating on Play Store?

#### Aaron
3. FANG, which company made the best apps?
4. Top singapore company apps?
5. App rating distribution
6.  Rating VS size/price band/free or paid/installs
7.  Pricing trend, how to price your app? (Swarmplot)
8.  Number of reviews VS Number of downloads

How to get "High" Rating on Play Store?
Market breakdown by game category, which category has most downloads?